## 86. Funciones definidas por el usuario:

In [2]:
%load_ext autoreload
%autoreload 2

import os
path = os.environ.get("HOME")
os.chdir(f"{path}/github/pyspark-hand-on")
from src.utils.utils import spark_session

spark = spark_session()

25/11/04 23:43:13 WARN Utils: Your hostname, personal-computer resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
25/11/04 23:43:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/04 23:43:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/04 23:43:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [23]:
def cubo(n):
    return n**3

In [24]:
from pyspark.sql.types import LongType, StringType
from pyspark.sql.functions import udf

In [25]:
## Registrar función como udf:
spark.udf.register("cubo", cubo, LongType())

25/11/05 00:03:46 WARN SimpleFunctionRegistry: The function cubo replaced a previously registered function.


<function __main__.cubo(n)>

In [26]:
spark.range(1, 10).createOrReplaceTempView("df_temp")

In [28]:
spark.sql("SELECT id, cubo(id) AS cubo FROM df_temp").show()

+---+----+
| id|cubo|
+---+----+
|  1|   1|
|  2|   8|
|  3|  27|
|  4|  64|
|  5| 125|
|  6| 216|
|  7| 343|
|  8| 512|
|  9| 729|
+---+----+



#### 2° forma de usar udf:

In [29]:
def bienvenida(nombre):
    return "Hola {}".format(nombre)

In [30]:
bienvenida_udf = udf(lambda x: bienvenida(x), StringType())

In [31]:
## Creacion de dataframe:
df_name = spark.createDataFrame([("Jose",), ("Julia",)], ["nombre"])

In [32]:
df.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- pago: long (nullable = true)



In [33]:
df.show(truncate=False)

+------+----+
|nombre|pago|
+------+----+
|Jose  |1   |
|Julia |2   |
|Katia |1   |
|NULL  |3   |
|Raul  |3   |
+------+----+



In [15]:
from pyspark.sql.functions import col

In [34]:
df.select(
    col("nombre"),
    bienvenida_udf(col("nombre")).alias("bienvenida_nombre")
).show()

+------+-----------------+
|nombre|bienvenida_nombre|
+------+-----------------+
|  Jose|        Hola Jose|
| Julia|       Hola Julia|
| Katia|       Hola Katia|
|  NULL|        Hola None|
|  Raul|        Hola Raul|
+------+-----------------+



#### 3° Forma de crear un udf:

In [35]:
@udf(returnType=StringType())
def mayuscula(s):
    return s.upper()

In [36]:
df_name.select(
    col("nombre"),
    mayuscula(col("nombre")).alias("nombre")
).show()

+------+------+
|nombre|nombre|
+------+------+
|  Jose|  JOSE|
| Julia| JULIA|
+------+------+



#### UDF de Pandas:

In [38]:
import pandas as pd
from pyspark.sql.functions import pandas_udf

In [40]:
def cubo_pandas(series:pd.Series)->pd.Series:
    return series**3

In [42]:
cubo_udf_pd = pandas_udf(cubo_pandas, returnType=LongType())

In [43]:
x = pd.Series([1,2,3])

In [44]:
print(cubo_pandas(x))

0     1
1     8
2    27
dtype: int64


In [45]:
df2 = spark.range(5)

In [46]:
df2.select(
    col("id"),
    cubo_udf_pd(col("id")).alias("cubo_pandas")
).show()

+---+-----------+
| id|cubo_pandas|
+---+-----------+
|  0|          0|
|  1|          1|
|  2|          8|
|  3|         27|
|  4|         64|
+---+-----------+

